In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
L = 1.0          # Length of the domain
Nx = 50         # Number of grid points
dx = L / (Nx - 1)
dt = 0.0005      # Time step
time_steps = 100 # Number of time iterations

# Thermal conductivity (Quadratic dependence)
def k(T):
    k0, k1, k2 = 1.0, 0.01, 0.001
    return k0 + k1 * T + k2 * T**2

# Initial condition
T = np.ones(Nx) * 300  # Uniform initial temperature

# Boundary conditions
T[0] = 400  # Fixed temperature at left boundary
T[-1] = 300 # Fixed temperature at right boundary

# Time stepping loop
for _ in range(time_steps):
    T_new = np.copy(T)
    
    # Compute temperature update using standard finite difference
    for i in range(1, Nx - 1):
        k_i = k(T[i])
        T_new[i] = T[i] + dt * k_i * (T[i+1] - 2*T[i] + T[i-1]) / dx**2
    
    T = np.copy(T_new)

# Plot results
plt.plot(np.linspace(0, L, Nx), T, label="Standard FD")
plt.xlabel("x")
plt.ylabel("Temperature")
plt.title("Temperature Distribution (Standard FD)")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_banded

# Constants
L = 1.0          # Length of the domain
Nx = 100         # Number of grid points
dx = L / (Nx - 1)
dt = 0.01        # Time step
time_steps = 50  # Number of time iterations

# Thermal conductivity (Quadratic dependence)
def k(T):
    k0, k1, k2 = 1.0, 0.01, 0.001
    return k0 + k1 * T + k2 * T**2

# Initial condition
T = np.ones(Nx) * 300

# Boundary conditions
T[0] = 400  # Fixed temperature at left boundary
T[-1] = 300 # Fixed temperature at right boundary

# Implicit matrix setup for conservative formulation
for _ in range(time_steps):
    A = np.zeros((3, Nx))  # Tridiagonal system: Lower, Main, Upper diagonals
    b = np.zeros(Nx)

    for i in range(1, Nx - 1):
        k_ip = (k(T[i]) + k(T[i+1])) / 2  # Interpolated at i+1/2
        k_im = (k(T[i]) + k(T[i-1])) / 2  # Interpolated at i-1/2
        
        alpha_p = dt * k_ip / dx**2
        alpha_m = dt * k_im / dx**2

        A[1, i] = 1 + alpha_p + alpha_m  # Main diagonal
        A[0, i] = -alpha_m               # Lower diagonal
        A[2, i] = -alpha_p               # Upper diagonal

        b[i] = T[i]  # Right-hand side

    # Apply boundary conditions
    A[1, 0] = 1
    A[1, -1] = 1
    b[0] = 400
    b[-1] = 300

    # Solve the tridiagonal system
    T = solve_banded((1, 1), A, b)

# Plot results
plt.plot(np.linspace(0, L, Nx), T, label="Implicit Conservative FD", linestyle="dashed")
plt.xlabel("x")
plt.ylabel("Temperature")
plt.title("Temperature Distribution (Implicit Conservative FD)")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_banded

# Parameters
L = 100         # Length of domain
Nx = 200        # Number of grid points
dx = L / Nx     # Grid spacing
dt = 0.01       # Time step
M = 1.0         # Mobility
kappa = 1.0     # Gradient energy coefficient
A = 1.0         # Double-well potential coefficient
time_steps = 500  # Number of iterations

# Discretization
x = np.linspace(0, L, Nx, endpoint=False)  # Space grid
c = np.random.uniform(0.4, 0.6, Nx)  # Initial condition (random phase separation)

# Periodic Laplacian function
def laplacian(u):
    return (np.roll(u, -1) - 2 * u + np.roll(u, 1)) / dx**2

# Time evolution
for _ in range(time_steps):
    # Compute chemical potential mu = f'(c) - kappa * laplacian(c)
    mu = A * (2 * c * (1 - c) * (1 - 2 * c)) - kappa * laplacian(c)
    
    # Update composition using Cahn-Hilliard equation
    c += dt * M * laplacian(mu)
    
    # Enforce periodic boundary conditions (implicitly handled by np.roll)

# Plot final profile
plt.plot(x, c, label="Final composition profile")
plt.xlabel("x")
plt.ylabel("c(x)")
plt.title("1D Cahn-Hilliard Simulation with Periodic BCs")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Constants
L = 1.0          # Length of the domain
Nx = 100         # Number of grid points
dx = L / (Nx - 1)
dt = 0.0001      # Time step
time_steps = 100 # Number of time iterations

# Thermal conductivity (Quadratic dependence)
def k(T):
    k0, k1, k2 = 1.0, 0.01, 0.001
    return k0 + k1 * T + k2 * T**2

# Initial condition
T = np.ones(Nx) * 300  # Uniform initial temperature

# Boundary conditions
T[0] = 400  # Fixed temperature at left boundary
T[-1] = 300 # Fixed temperature at right boundary

# Time stepping loop
for _ in range(time_steps):
    T_new = np.copy(T)

    # Compute fluxes at interfaces
    for i in range(1, Nx - 1):
        k_i_p_half = (k(T[i]) + k(T[i+1])) / 2  # Arithmetic mean
        k_i_m_half = (k(T[i]) + k(T[i-1])) / 2  # Arithmetic mean

        flux_right = -k_i_p_half * (T[i+1] - T[i]) / dx
        flux_left = -k_i_m_half * (T[i] - T[i-1]) / dx

        # Update temperature using conservative form
        T_new[i] = T[i] + dt / dx * (flux_left - flux_right)

    T = np.copy(T_new)

# Plot results
plt.plot(np.linspace(0, L, Nx), T, label="Conservative FD", linestyle="dashed")
plt.xlabel("x")
plt.ylabel("Temperature")
plt.title("Temperature Distribution (Conservative FD)")
plt.legend()
plt.show()
